In [124]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

In [125]:
import pandas as pd
from datasets import Dataset, DatasetDict
import torchaudio

# Load Excel file
data = pd.read_excel('Aya_data.xlsx')
data['Transcription'] = data['Transcription'].str.strip()
data = data.dropna(subset=['Transcription'])
data = data.iloc[:, :3]
data['Transcription'] = data['Transcription'].str.replace('ث', 'س')
data = data.head(1850)

# Assuming your columns are 'Segment' and 'Transcription'
# data['path'] = 'Resampled_audio/' + data['Segment'] + '.wav'
data['audio'] = 'Resampled_audio/' + data['Segment'] + '.wav'

def load_audio(file_path):
    waveform, sample_rate = torchaudio.load(file_path)
    return {
        'path': file_path,
        'array': waveform.squeeze().numpy(),
        'sampling_rate': sample_rate
    }

# Apply the load_audio function to each audio file in your dataset
data['audio'] = data['audio'].apply(load_audio)

# Rename columns to match the required format
data = data.rename(columns={'Transcription': 'sentence'})
data = data.drop(columns=['Segment','ID'])

# Convert to datasets format
dataset = Dataset.from_pandas(data)

# Split the dataset into train and test
dataset = dataset.train_test_split(test_size=0.1)

# Add the dataset to DatasetDict
dataset_dict = DatasetDict({
    'train': dataset['train'],
    'test': dataset['test']
})

# Verify the format
print(dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['sentence', 'audio', '__index_level_0__'],
        num_rows: 1662
    })
    test: Dataset({
        features: ['sentence', 'audio', '__index_level_0__'],
        num_rows: 185
    })
})


In [126]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")


In [127]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="arabic", task="transcribe")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [128]:
input_str = dataset_dict["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


Input:                 لأنو أنقذ الحي هيدا الولد لما الحي عرف يحبو
Decoded w/ special:    <|startoftranscript|><|ar|><|transcribe|><|notimestamps|>لأنو أنقذ الحي هيدا الولد لما الحي عرف يحبو<|endoftext|>
Decoded w/out special: لأنو أنقذ الحي هيدا الولد لما الحي عرف يحبو
Are equal:             True


In [129]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="arabic", task="transcribe")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [130]:
dataset_dict["train"][0]

{'sentence': 'لأنو أنقذ الحي هيدا الولد لما الحي عرف يحبو',
 'audio': {'array': [0.0008544921875,
   0.00103759765625,
   0.00067138671875,
   0.002288818359375,
   0.00286865234375,
   0.0028076171875,
   0.002960205078125,
   0.001739501953125,
   0.000274658203125,
   -3.0517578125e-05,
   0.0,
   -0.00079345703125,
   -0.00128173828125,
   -0.000579833984375,
   0.00030517578125,
   0.000885009765625,
   0.000274658203125,
   0.000457763671875,
   0.00091552734375,
   0.0,
   -0.001129150390625,
   -0.0009765625,
   -0.000457763671875,
   -0.000335693359375,
   -0.000213623046875,
   -0.000396728515625,
   -0.0003662109375,
   -0.000762939453125,
   -0.0015869140625,
   -0.00213623046875,
   -0.001007080078125,
   0.000518798828125,
   0.0009765625,
   0.00189208984375,
   0.0032958984375,
   0.00341796875,
   0.00225830078125,
   0.000885009765625,
   -0.00091552734375,
   -0.00140380859375,
   -0.00103759765625,
   -0.000732421875,
   0.0003662109375,
   0.000457763671875,
   -6.

In [131]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch


In [132]:
dataset_dict = dataset_dict.map(prepare_dataset,batch_size=4, remove_columns=dataset_dict.column_names["train"], num_proc=1)


Map:   0%|          | 0/1662 [00:00<?, ? examples/s]

Map:   0%|          | 0/185 [00:00<?, ? examples/s]

In [142]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")


In [143]:
model.generation_config.language = "arabic"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None


In [144]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [145]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)


In [146]:
import evaluate

metric = evaluate.load("wer")



In [147]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [152]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./lebanese_dialect_asr",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=100,
    max_steps=800,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=448,
    save_steps=10,
    eval_steps=10,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)


In [153]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor, 
)


max_steps is given, it will override any value given in num_train_epochs


In [154]:
trainer.train()


  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


{'eval_loss': 1.4717897176742554, 'eval_wer': 70.10078387458006, 'eval_runtime': 994.6711, 'eval_samples_per_second': 0.186, 'eval_steps_per_second': 0.024, 'epoch': 0.1}


c:\Users\PcUser\anaconda3\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


...

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


{'eval_loss': 0.5087454915046692, 'eval_wer': 40.42553191489361, 'eval_runtime': 997.0036, 'eval_samples_per_second': 0.186, 'eval_steps_per_second': 0.024, 'epoch': 5.77}


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


{'train_runtime': 152738.8718, 'train_samples_per_second': 0.063, 'train_steps_per_second': 0.004, 'train_loss': 0.257595775226752, 'epoch': 5.77}


TrainOutput(global_step=600, training_loss=0.257595775226752, metrics={'train_runtime': 152738.8718, 'train_samples_per_second': 0.063, 'train_steps_per_second': 0.004, 'total_flos': 2.7675339890688e+18, 'train_loss': 0.257595775226752, 'epoch': 5.769230769230769})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
kwargs = {
    # "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Whisper",  # a 'pretty' name for the training dataset
    "dataset_args": "config: ar, split: test",
    "language": "ar",
    "model_name": "Whisper Small Arabic",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [ ]:
trainer.push_to_hub(**kwargs)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AyaOsta/lebanese_dialect_asr/commit/8ab3bcaa14ec0e9d01df548834a5593686e9b17d', commit_message='End of training', commit_description='', oid='8ab3bcaa14ec0e9d01df548834a5593686e9b17d', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("lebanese_dialect_asr")

README.md:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

c:\Users\PcUser\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PcUser\.cache\huggingface\hub\models--AyaOsta--lebanese_dialect_asr. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/AyaOsta/lebanese_dialect_asr/commit/5ff8487ea80e47d94b1d070d63c0b21e9339174f', commit_message='Upload tokenizer', commit_description='', oid='5ff8487ea80e47d94b1d070d63c0b21e9339174f', pr_url=None, pr_revision=None, pr_num=None)

In [1]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="AyaOsta/lebanese_dialect_asr")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text




config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

c:\Users\PcUser\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PcUser\.cache\huggingface\hub\models--AyaOsta--lebanese_dialect_asr. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
iface = gr.Interface(
    fn=transcribe, 
    inputs=gr.Audio(type="filepath"), 
    outputs="text",
    title="Lebanese Speech Recognition Model",
    description="Realtime demo for Arabic speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


c:\Users\PcUser\anaconda3\Lib\site-packages\transformers\models\whisper\generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
